In [1]:
#Intento de Solucion del desafio  15k
#que NO logra solucionarlo, una que falta una idea fundamental, una chispa, un Momento Eureka
#pero crea estructura sobre la cual trabajar


#limpio la memoria
rm( list=ls() )
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,602652,32.2,1286600,68.8,1286600,68.8
Vcells,1082303,8.3,8388608,64.0,1632034,12.5


In [2]:
require("data.table")

Loading required package: data.table



In [3]:
ftirar  <- function( prob, qty )
{
  return(  sum( runif(qty) < prob ) )
}

In [4]:
#variables globales que usan las funciones gimnasio_xxxx
GLOBAL_jugadores  <- c()
GLOBAL_tiros_total  <- 0

In [5]:
#Crea el juego
#a cada jugador se le pone un numero de 1 a 100 en la espalda
#debajo de ese numero esta el indice_de_enceste  que NO puede ser visto por el cazatalentos
gimnasio_init  <- function() 
{
  GLOBAL_jugadores  <<- sample( c( (501:599 ) / 1000 , 0.7 ) )
  GLOBAL_tiros_total  <<- 0
}

In [6]:
#se le pasa un vector con los IDs de los jugadores y la cantidad de tiros a realizar
#devuelve en un vector cuantos aciertos tuvo cada jugador
gimnasio_tirar  <- function(  pids,  pcantidad )
{
  GLOBAL_tiros_total  <<-  GLOBAL_tiros_total + length( pids )*pcantidad
  res  <- mapply(  ftirar, GLOBAL_jugadores[pids], pcantidad )

  return( res )
}

In [7]:
#El cazatalentos decide a que jugador llevarse
#devuelve la cantidad de tiros libres y si le acerto al verdadero_mejor o no
gimnasio_veredicto  <- function( pid )
{
  return( list("tiros_total"= GLOBAL_tiros_total, 
               "acierto"=     as.integer( GLOBAL_jugadores[pid]==0.7) ))
}
#------------------------------------------------------------------------------

In [8]:
Estrategia_A  <- function()
{
    #inicializo el juego
    gimnasio_init()
    #Esta el la planilla del cazatalentos
    planilla_cazatalentos  <- data.table( "id" = 1:100 )
    #Ronda 1  ------------------------------------------------------
    #tiran los 100 jugadores es decir 1:100  50 tiros libres cada uno
    ids_juegan1  <- 1:100   #los jugadores que participan en la ronda,
    
    planilla_cazatalentos[ ids_juegan1,  tiros1 := 10 ]  #registro en la planilla que tiran 90 tiros
    
    #Hago que tiren
    resultado1  <- gimnasio_tirar( ids_juegan1, 10)
    planilla_cazatalentos[ ids_juegan1,  aciertos1 := resultado1 ]  #registro en la planilla
    
    #Ronda 2  ------------------------------------------------------
    #tiran los 100 jugadores es decir 1:100  50 tiros libres cada uno
    ids_juegan2  <- 1:100   #los jugadores que participan en la ronda,

    planilla_cazatalentos[ ids_juegan2,  tiros2 := 10 ]  #registro en la planilla que tiran 90 tiros
    
    #Hago que tiren
    resultado2  <- gimnasio_tirar( ids_juegan2, 10)
    planilla_cazatalentos[ ids_juegan2,  aciertos2 := resultado2 ]  #registro en la planilla
    
    #Ronda 22  ------------------------------------------------------
    #tiran los 100 jugadores es decir 1:100  50 tiros libres cada uno
    ids_juegan22  <- 1:100   #los jugadores que participan en la ronda,

    planilla_cazatalentos[ ids_juegan22,  tiros22 := 10 ]  #registro en la planilla que tiran 90 tiros
    
    #Hago que tiren
    resultado22  <- gimnasio_tirar( ids_juegan22, 10)
    planilla_cazatalentos[ ids_juegan22,  aciertos22 := resultado22 ]  #registro en la planilla
    
    planilla_cazatalentos$aciertos1y2 <- planilla_cazatalentos$aciertos1 + planilla_cazatalentos$aciertos2 + planilla_cazatalentos$aciertos22
    
    #Ronda 3 -------------------------------------------------------
    #los mejores 10 jugadores tiran 415 tiros cada uno
    #aciertos1y2 <- (aciertos1 + aciertos2)
    data_mod <- planilla_cazatalentos[order(planilla_cazatalentos$aciertos1y2, decreasing = TRUE), ] #Orderno de mayor a menor aciertos1y2
    top10  <- data_mod[ , head(.SD, 10) ]
    min  <- min(top10$aciertos1y2)
    ids_juegan3 <- planilla_cazatalentos[ ids_juegan1 ][aciertos1y2 >= min, id ]
    
      return( ids_juegan3 )
}

In [8]:
# NO CORRERRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
Estrategia_A  <- function()
{
  #Estrategia
  #En la primer ronda se hace tirar 90 tiros libres a cada uno de los 100 jugadores ( se gastan 9000 tiros )
  #Se eligen a la mejor mitad de primer ronda( se descarta a la otra mitad)
  #En la segunda ronda, a la mejor mitad de la primera se los hace tirar 400 tiros a cada uno
  #Se elige el mejor jugador de la segunda ronda

  gimnasio_init()

  #Esta el la planilla del cazatalentos
  #el id es el numero que tiene en la espalda cada jugador
  planilla_cazatalentos  <- data.table( "id"= 1:100 )

  #Ronda 1  ------------------------------------------------------
  #tiran los 100 jugadores es decir 1:100   90  tiros libres cada uno
  ids_juegan1  <- 1:100   #los jugadores que participan en la ronda,

  planilla_cazatalentos[ ids_juegan1,  tiros1 := 90 ]  #registro en la planilla que tiran 90 tiros

  #Hago que tiren
  resultado1  <- gimnasio_tirar( ids_juegan1, 90)
  planilla_cazatalentos[ ids_juegan1,  aciertos1 := resultado1 ]  #registro en la planilla

  #Ronda 2 -------------------------------------------------------
  #A la mitad mejor la hago tirar 400 tiros cada uno
  #La mediana siempre parte a un conjunto en dos partes de igual cantidad
  mediana  <- planilla_cazatalentos[ ids_juegan1, median(aciertos1) ]
  ids_juegan2  <- planilla_cazatalentos[ ids_juegan1 ][ aciertos1 >= mediana, id ]

  planilla_cazatalentos[ ids_juegan2,  tiros2 := 400 ]  #registro en la planilla que tiran 400 tiros
  resultado2  <- gimnasio_tirar( ids_juegan2, 400)
  planilla_cazatalentos[ ids_juegan2,  aciertos2 := resultado2 ]  #registro en la planilla

  #Epilogo
  #El cazatalentos toma una decision, elige al que mas aciertos tuvo en la ronda2
  pos_mejor <-  planilla_cazatalentos[ , which.max(aciertos2) ]
  id_mejor  <-  planilla_cazatalentos[ pos_mejor, id ]

  #Finalmente, la hora de la verdadero_mejor
  #Termino el juego
  veredicto  <- gimnasio_veredicto( id_mejor )
  
  return( veredicto )
}
#------------------------------------------------------------------------------

In [11]:
#Aqui hago la Estimacion Montecarlo del porcentaje de aciertos que tiene la estrategia A

tabla_veredictos  <- data.table( ids_juegan3=integer() )

In [13]:
for( experimento  in  1:10000 )
{
  if( experimento %% 1000 == 0 )  cat( experimento, " ")  #desprolijo, pero es para saber por donde voy

  ids_juegan3  <- Estrategia_A()
  
  tabla_veredictos  <- rbind( ids_juegan3 )
}

cat("\n")

1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  


In [14]:
# FS: Seteo donde quiero que me baje el csv
setwd("C:/Users/flore/OneDrive/Escritorio/Flor/Maestria/DMEyF")

In [16]:
# FS: Bajo el csv
write.table(tabla_veredictos,"tabla_veredictos en txt.txt")

In [11]:
tiros_total  <-  tabla_veredictos[  , max( tiros_total) ]
tasa_eleccion_correcta  <-  tabla_veredictos[  , mean( acierto) ]

In [12]:
tiros_total
tasa_eleccion_correcta

#Esta estrategia elije al verdadero_mejor el 99% de las veces
#pero lamentablemente necesita de un total de 36600   tiros libresa

[1] 35400

[1] 0.9904